# 数据预处理
### 我们首先遍历下语料文件下的所有文件夹，获得不同类别的新闻以及对应的类别标签，代码如下所示

In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

C:\Users\two\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
BASE_DIR = "data\\"
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')

In [3]:
# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


### 之后，我们可以新闻样本转化为神经网络训练所用的张量。所用到的Keras库是keras.preprocessing.text.Tokenizer和keras.preprocessing.sequence.pad_sequences。代码如下所示

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

C:\Users\two\Anaconda3\lib\site-packages\keras\preprocessing\text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


# Embedding layer设置
### 接下来，我们从GloVe文件中解析出每个词和它所对应的词向量，并用字典的方式存储

In [6]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


### 此时，我们可以根据得到的字典生成上文所定义的词向量矩阵

In [7]:
EMBEDDING_DIM = 100

In [8]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [9]:
embedding_matrix.shape

(174075, 100)

### 现在我们将这个词向量矩阵加载到Embedding层中，注意，我们设置trainable=False使得这个编码层不可再训练。

In [10]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### 一个Embedding层的输入应该是一系列的整数序列，比如一个2D的输入，它的shape值为(samples, indices)，也就是一个samples行，indeces列的矩阵。每一次的batch训练的输入应该被padded成相同大小（尽管Embedding层有能力处理不定长序列，如果你不指定数列长度这一参数） dim). 所有的序列中的整数都将被对应的词向量矩阵中对应的列（也就是它的词向量）代替,比如序列[1,2]将被序列[词向量[1],词向量[2]]代替。这样，输入一个2D张量后，我们可以得到一个3D张量。

# 训练1D卷积
### 最后，我们可以使用一个小型的1D卷积解决这个新闻分类问题。

In [11]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         17407500  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048     
__________

In [13]:
# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size=128, verbose=1)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 238s 15ms/step - loss: 2.4208 - acc: 0.2120 - val_loss: 1.7519 - val_acc: 0.3936
Epoch 2/10
15998/15998 [==============================] - 240s 15ms/step - loss: 1.5493 - acc: 0.4572 - val_loss: 1.2759 - val_acc: 0.5621
Epoch 3/10
15998/15998 [==============================] - 244s 15ms/step - loss: 1.2055 - acc: 0.5835 - val_loss: 1.1972 - val_acc: 0.5734
Epoch 4/10
15998/15998 [==============================] - 246s 15ms/step - loss: 0.9902 - acc: 0.6607 - val_loss: 1.0236 - val_acc: 0.6594
Epoch 5/10
15998/15998 [==============================] - 243s 15ms/step - loss: 0.8370 - acc: 0.7185 - val_loss: 1.0193 - val_acc: 0.6732
Epoch 6/10
15998/15998 [==============================] - 244s 15ms/step - loss: 0.7125 - acc: 0.7575 - val_loss: 0.9866 - val_acc: 0.6919
Epoch 7/10
15998/15998 [==============================] - 250s 16ms/step - loss: 0.6059 - acc: 0.7922 - val_loss: 0.9

In [19]:
word_embed_layer = model.layers[1]
weights = word_embed_layer.get_weights()[0]

In [20]:
weights[0].shape

(100,)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
QUERY1 = 'good'
QUERY2 = 'great'

print("Finetune:", cosine_similarity(weights[word_index[QUERY1]].reshape(-1, EMBEDDING_DIM),
                                   weights[word_index[QUERY2]].reshape(-1, EMBEDDING_DIM)))

Finetune: [[0.7592797]]


In [73]:
QUERY1 = 'good'
QUERY2 = 'rice'

print("Finetune:", cosine_similarity(weights[word_index[QUERY1]].reshape(-1, EMBEDDING_DIM),
                                   weights[word_index[QUERY2]].reshape(-1, EMBEDDING_DIM)))

Finetune: [[0.4321299]]


In [83]:
with open("transfer_vectors.txt", "w", encoding='utf8') as f:
    f.write(str(len(word_index)))
    f.write(' ')
    f.write(str(EMBEDDING_DIM))
    f.write('\n')
    for word, ind in word_index.items():
        if ind%50000 == 0:
            print('Processing : {:.2f}%'.format(100*ind/len(word_index)))
        f.write(word)
        for vec in weights[ind]:
            f.write(" ")
            f.write(str(vec))
        if ind < len(word_index):
            f.write('\n')
    print('Processing : 100%')

Processing : 28.72%
Processing : 57.45%
Processing : 86.17%
Processing : 100%


In [91]:
import gensim
from pprint import pprint

In [84]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("transfer_vectors.txt", binary=False)

In [94]:
WORD = "finance"

pprint(word2vec.most_similar(WORD))

[('banking', 0.7193477153778076),
 ('foreign', 0.7031714916229248),
 ('minister', 0.6959875822067261),
 ('financial', 0.6851174831390381),
 ('commerce', 0.6667627096176147),
 ('investment', 0.6605798602104187),
 ('reform', 0.6556479334831238),
 ('government', 0.6550486087799072),
 ('fund', 0.6535806655883789),
 ('affairs', 0.6535346508026123)]
